In [7]:
import json
from collections import Counter

import pandas as pd

In [2]:
with open('data/dataset.json', 'r') as f:
    data = json.load(f)

In [4]:
target_sents = [sample['predict'] for sample in data]

In [9]:
df = pd.json_normalize(target_sents)

In [17]:
df.head()

,text,midas,entity.text,entity.label
0,"Yes like good riddance, in my heart of hearts ...",opinion,good riddance,videoname
1,"I used to in my childhood but not any more, I ...",opinion,inspector Gadget,videoname
2,"Yes, 20 quintillion, can you believe that?Wow",yes_no_question,20 quintillion,number
3,"Yes indeed, interesting facts that will give m...",opinion,weekend,date
4,hmm that seems dumb like its just playing with...,yes_no_question,Teddy Roosevelt,person


In [21]:
def normalize_text(row):
    return row['text'].replace(row['entity.text'], row['entity.label'].upper())

In [23]:
df['normalized_text'] = df[['text', 'entity.text', 'entity.label']].apply(normalize_text, axis=1)

In [48]:
rare_midas = ['other_answers', 'appreciation', 'dev_command']
rare_entity = ['bookname', 'venue', 'ordinal']
df_common_midas = df[~df.midas.isin(rare_midas)]
df_rare_midas = df[df.midas.isin(rare_midas)]
df_common_entity = df[~df['entity.label'].isin(rare_entity)]

In [53]:
common_sampled_by_midas = df_common_midas.groupby('midas', group_keys=False).apply(lambda x: x.sample(10))
common_sampled_by_entity = df_common_entity.groupby('entity.label', group_keys=False).apply(lambda x: x.sample(10))
rare_sampled_by_midas = df_rare_midas.groupby('midas', group_keys=False).apply(lambda x: x.sample(4))

In [63]:
pd.concat([common_sampled_by_midas[['midas', 'normalized_text']], 
           rare_sampled_by_midas[['midas', 'normalized_text']]]).to_csv('data/midas_sample.tsv', sep='\t', index=False)

In [64]:
common_sampled_by_entity[['entity.label', 'normalized_text']].to_csv('data/entity_sample.tsv', sep='\t', index=False)